In [1]:
from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core import Environment
from azureml.core import Dataset, Datastore

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig

In [2]:
from azureml.core.compute import ComputeInstance

In [3]:
workspace = Workspace.from_config()

In [4]:
instance = ComputeTarget(workspace=workspace, name='gpu-v100-low')

In [5]:
dataset = Dataset.get_by_name(workspace, name='recursionbio')

In [6]:
dataset

{
  "registration": {
    "id": "ee33756c-ab30-45ec-adf9-5b93629bbbd8",
    "name": "recursionbio",
    "version": 2,
    "workspace": "Workspace.create(name='bioworks-azureml', subscription_id='73f2271c-beee-4233-b5dd-202b520bf0db', resource_group='mslearn-azureml')"
  }
}

In [7]:
env_list = Environment.list(workspace)

In [8]:
env_list.keys()

dict_keys(['AzureML-VowpalWabbit-8.8.0', 'AzureML-PyTorch-1.3-CPU', 'AzureML-Pytorch1.7-Cuda11-OpenMpi4.1.0-py36', 'AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36', 'AzureML-TensorFlow2.4-Cuda11-OpenMpi4.1.0-py36', 'AzureML-Triton', 'AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference', 'AzureML-minimal-ubuntu18.04-py37-cpu-inference', 'AzureML-mlflow-ubuntu18.04-py37-cpu-inference', 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference', 'AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference', 'AzureML-pytorch-1.7-ubuntu18.04-py37-cpu-inference', 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu', 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11.0.3-gpu-inference', 'AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu', 'AzureML-tensorflow-1.15-ubuntu18.04-py37-cpu-inference', 'AzureML-pytorch-1.6-ubuntu18.04-py37-cpu-inference', 'AzureML-xgboost-0.9-ubuntu18.04-py37-cpu-inference', 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu', 'AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu', 'AzureML-p

In [9]:
tf_env = Environment.get(workspace=workspace, name='AzureML-TensorFlow-2.3-GPU')
tf_env = tf_env.clone(new_name='regbio-tf-env')

In [10]:
tf_env.python.conda_dependencies.add_conda_package('scikit-learn')
tf_env.python.conda_dependencies.add_conda_package('scipy')
tf_env.python.conda_dependencies.add_conda_package('matplotlib')

In [11]:
tf_env.python.conda_dependencies.add_pip_package('horovod==0.19.5')
tf_env.python.conda_dependencies.add_pip_package('retry')
tf_env.python.conda_dependencies.add_pip_package('tqdm')

In [12]:
tf_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04:20211124.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "regbio-tf-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            

In [14]:
train_scr = ScriptRunConfig(
    source_directory='./scripts',
    script='training.py',
    arguments=['--data-path', dataset.as_mount(),
               '--epochs', 100,
               '--batch', 24,
               ],
    compute_target=instance,
    environment=tf_env
)

In [15]:
train_scr

In [16]:
run = Experiment(workspace=workspace, name='test-training-run').submit(train_scr)

In [17]:
run.wait_for_completion(show_output=True)

RunId: test-training-run_1638393296_731acea0
Web View: https://ml.azure.com/runs/test-training-run_1638393296_731acea0?wsid=/subscriptions/73f2271c-beee-4233-b5dd-202b520bf0db/resourcegroups/mslearn-azureml/workspaces/bioworks-azureml&tid=52603c87-c32e-48b2-b2c4-e366f50b2505

Streaming azureml-logs/20_image_build_log.txt

2021/12/01 21:15:00 Downloading source code...
2021/12/01 21:15:01 Finished downloading source code
2021/12/01 21:15:02 Creating Docker network: acb_default_network, driver: 'bridge'
2021/12/01 21:15:04 Successfully set up Docker network: acb_default_network
2021/12/01 21:15:04 Setting up Docker configuration...
2021/12/01 21:15:05 Successfully set up Docker configuration
2021/12/01 21:15:05 Logging in to registry: 2e7eaafe1ee440358c1d803ca534b890.azurecr.io
2021/12/01 21:15:06 Successfully logged into 2e7eaafe1ee440358c1d803ca534b890.azurecr.io
2021/12/01 21:15:06 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt.",
        "messageFormat": "Image build failed. For more details, check log file {ArtifactPath}.",
        "messageParameters": {
            "ArtifactPath": "azureml-logs/20_image_build_log.txt"
        },
        "details": [],
        "innerError": {
            "code": "BadArgument",
            "innerError": {
                "code": "ImageBuildFailure"
            }
        }
    },
    "correlation": {
        "operation": "bb8137dfbd2d114db67feb2157381f02",
        "request": "775342238e1e977d"
    },
    "environment": "northeurope",
    "location": "northeurope",
    "time": "2021-12-01T21:20:14.158799Z",
    "componentName": "execution-worker"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"e